# thought dumping
might try swap to langchain 
or use anthropic api to merge search results into comprehensive report

In [ ]:
!pip install google-api-python-client
from googleapiclient.discovery import build
import concurrent.futures
import ast
import os
import re
from llama_cpp import Llama

#ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")  # Replace with your Anthropic API key
GOOGLE_API_KEY = os.getenv("GOOGLE_SEARCH")  # Replace with your Google Search API key
GOOGLE_CSE_ID = os.getenv("GOOGLE_SEARCH_ENGINE_ID")  # Replace with your Custom Search Engine ID

In [ ]:
# borked

from googleapiclient.discovery import build
import concurrent.futures
import os
import re
from llama_cpp import Llama
import time

GOOGLE_API_KEY = os.getenv("GOOGLE_SEARCH")  # Replace with your Google Search API key
GOOGLE_CSE_ID = os.getenv("GOOGLE_SEARCH_ENGINE_ID")  # Replace with your Custom Search Engine ID

def remove_first_line(test_string):
    if test_string.startswith("Here") and test_string.split("\n")[0].strip().endswith(":"):
        return re.sub(r'^.*\n', '', test_string, count=1)
    return test_string

def generate_text(llm, prompt, max_tokens=2000, temperature=0.7, max_retries=3, retry_delay=1):
    for i in range(max_retries):
        try:
            output = llm(prompt, max_tokens=max_tokens, temperature=temperature, stop=[], echo=False)
            response = remove_first_line(output['choices'][0]['text'].strip())
            print(f"Response: {response}")
            match = re.search(r'\[(.*?)\]', response)
            if match:
                try:
                    items = [item.strip().strip('"').strip("'") for item in match.group(1).split(',')]
                    print(f"Items: {items}")
                    return items
                except (SyntaxError, ValueError) as e:
                    print(f"Error parsing items: {e}")
                    pass
        except Exception as e:
            print(f"Error generating text (attempt {i+1}/{max_retries}): {e}")
            if i < max_retries - 1:
                time.sleep(retry_delay)
    return []

def search_web(search_term, api_key, cse_id):
    try:
        service = build("customsearch", "v1", developerKey=api_key)
        return service.cse().list(q=search_term, cx=cse_id).execute()
    except Exception as e:
        print(f"Error searching web: {e}")
        return None

def chunk_prompt(llm, prompt, max_tokens):
    """Splits a prompt into chunks that fit within the model's max token limit."""
    prompt_bytes = prompt.encode('utf-8')  # Convert prompt to bytes
    prompt_tokens = llm.tokenize(prompt_bytes)
    chunks = []
    current_chunk = []
    for token in prompt_tokens:
        if len(current_chunk) + 1 <= max_tokens:
            current_chunk.append(token)
        else:
            chunks.append(llm.detokenize(current_chunk))
            current_chunk = [token]
    if current_chunk:
        chunks.append(llm.detokenize(current_chunk))
    return chunks

def process_chunks(llm, chunks, max_tokens):
    """Processes prompt chunks and combines the generated text."""
    results = []
    for chunk in chunks:
        result = generate_text(llm, chunk, max_tokens=max_tokens)
        if isinstance(result, list):
            results.extend(result)
        else:
            results.append(result)
    combined_result = " ".join(results)
    return combined_result

def generate_subtopic_report(llm, research_topic, subtopic, search_cache, max_workers=5):
    all_queries = []
    search_data = []
    print(f"Generating initial search queries for subtopic: {subtopic}...")
    initial_queries_prompt = f"Generate 5 search queries to gather information on the subtopic '{subtopic}' of the research topic '{research_topic}'. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
    initial_queries = generate_text(llm, initial_queries_prompt)
    all_queries.extend(initial_queries)
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        search_results = list(executor.map(lambda query: search_cache.setdefault(query, search_web(query, GOOGLE_API_KEY, GOOGLE_CSE_ID)), initial_queries))
    search_data.extend(search_results)
    print(f"Generating initial report for subtopic: {subtopic}...")
    report_prompt_1 = f"When writing your report, make it incredibly detailed, thorough, specific, and well-structured. Use Markdown for formatting. "
    report_prompt_2 = f"Analyze the following search data and generate a comprehensive report on the subtopic '{subtopic}' of the research topic '{research_topic}':\n\n{str(search_data)}"
    report_prompt = report_prompt_1 + report_prompt_2

    prompt_chunks = chunk_prompt(llm, report_prompt, max_tokens=1000)

    report = process_chunks(llm, prompt_chunks, max_tokens=1000)
    print(f"Analyzing report and generating additional searches for subtopic: {subtopic}...")
    analysis_prompt_1 = f"Analyze the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' and identify areas that need more detail or further information:\n\n{report}\n\n"
    analysis_prompt_2 = "---\n\nHere are all the search queries you have used so far for this subtopic:\n\n"
    analysis_prompt_3 = f"{str(all_queries)}\n\n---\n\nGenerate 3 new and unique search queries to fill in the gaps and provide more detail on the identified areas. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
    analysis_prompt = analysis_prompt_1 + analysis_prompt_2 + analysis_prompt_3
    additional_queries = generate_text(llm, analysis_prompt)
    all_queries.extend(additional_queries)
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        additional_search_results = list(executor.map(lambda query: search_cache.setdefault(query, search_web(query, GOOGLE_API_KEY, GOOGLE_CSE_ID)), additional_queries))
    search_data.extend(additional_search_results)
    print(f"Updating report with additional information for subtopic: {subtopic}...")
    update_prompt_1 = f"Update the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' by incorporating the new information from the additional searches. Keep all existing information... only add new information:\n\n{report}\n\n---\n\nAdditional search data:\n\n{str(additional_search_results)}\n\n---\n\n"
    update_prompt_2 = "Generate an updated report that includes the new information and provides more detail in the identified areas. Use Markdown for formatting."
    update_prompt = update_prompt_1 + update_prompt_2
    update_prompt_chunks = chunk_prompt(llm, update_prompt, max_tokens=1000)
    # Process each chunk separately and combine the results
    report = process_chunks(llm, update_prompt_chunks, max_tokens=1000)
    print(f"Final report generated for subtopic: {subtopic}!")
    return report

def generate_comprehensive_report(llm, research_topic, subtopic_reports):
    print("Generating comprehensive report...")
    comprehensive_report_prompt_1 = f"Generate a comprehensive report on the research topic '{research_topic}' by combining the following reports on various subtopics:\n\n{subtopic_reports}\n\n"
    comprehensive_report_prompt_2 = "---\n\nEnsure that the final report is well-structured, coherent, and covers all the important aspects of the research topic. Make sure that it includes EVERYTHING in each of the reports, in a better structured, more info-heavy manner. Nothing -- absolutely nothing, should be left out. If you forget to include ANYTHING from any of the previous reports, you will face the consequences. Include a table of contents. Leave nothing out. Use Markdown for formatting."
    comprehensive_report_prompt = comprehensive_report_prompt_1 + comprehensive_report_prompt_2
    comprehensive_report_prompt_chunks = chunk_prompt(llm, comprehensive_report_prompt, max_tokens=1000)
    comprehensive_report = process_chunks(llm, comprehensive_report_prompt_chunks, max_tokens=1000)
    return comprehensive_report

def is_valid_subtopic_list(subtopic_list, research_topic):
    if not isinstance(subtopic_list, list):
        print("Subtopic checklist is not a list.")
        return False
    for subtopic in subtopic_list:
        if not isinstance(subtopic, str):
            print(f"Subtopic '{subtopic}' is not a string.")
            return False
        if subtopic.lower() == research_topic.lower():
            print(f"Subtopic '{subtopic}' is the same as the research topic.")
            return False
    return True

def main(research_topic, model_path, max_tokens=1000, max_subtopics=5, max_workers=5):
    try:
        llm = Llama(model_path=model_path)
    except Exception as e:
        print(f"Error loading Llama model: {e}")
        return
    if not GOOGLE_API_KEY:
        print("Google Search API key not found. Please set the GOOGLE_SEARCH environment variable.")
        return
    if not GOOGLE_CSE_ID:
        print("Google Custom Search Engine ID not found. Please set the GOOGLE_SEARCH_ENGINE_ID environment variable.")
        return
    # Generate subtopic checklist
    subtopic_checklist_prompt = f"Generate a detailed checklist of subtopics to research for the topic '{research_topic}'. Return your checklist as a valid Python list, with each subtopic enclosed in single quotes and separated by commas. Maximum {max_subtopics} sub-topics. Start your response with ['"
    subtopic_checklist = generate_text(llm, subtopic_checklist_prompt)
    print(f"Generated subtopic checklist: {subtopic_checklist}")
    if not is_valid_subtopic_list(subtopic_checklist, research_topic):
        print("Invalid subtopic checklist generated. Please try again with a different research topic.")
        return
    print(f"Research Topic: {research_topic}")
    print(f"Subtopic Checklist: {subtopic_checklist}")
    # Generate reports for each subtopic
    search_cache = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        subtopic_reports = list(executor.map(lambda subtopic: generate_subtopic_report(llm, research_topic, subtopic, search_cache, max_workers=max_workers), subtopic_checklist))

    comprehensive_report = generate_comprehensive_report(llm, research_topic, "\n\n".join(subtopic_reports))
    print("Comprehensive report generated!")

    with open("comprehensive_report.txt", "w") as file:
        file.write(comprehensive_report)
    print("Comprehensive report saved as 'comprehensive_report.txt'.")

if __name__ == "__main__":
    research_topic = input("Enter the research topic: ")
    model_path = input("Enter the path to the Llama model file: ")
    main(research_topic, model_path, max_tokens=1000, max_subtopics=5, max_workers=5)

In [ ]:
# was working for ~7 minutes then reaches context window limit post-subtopic research feelsbadman
# !pip install google-api-python-client
from googleapiclient.discovery import build
import concurrent.futures
import os
import re
from llama_cpp import Llama
import time

GOOGLE_API_KEY = os.getenv("GOOGLE_SEARCH")  # Replace with your Google Search API key
GOOGLE_CSE_ID = os.getenv("GOOGLE_SEARCH_ENGINE_ID")  # Replace with your Custom Search Engine ID

def remove_first_line(test_string):
    if test_string.startswith("Here") and test_string.split("\n")[0].strip().endswith(":"):
        return re.sub(r'^.*\n', '', test_string, count=1)
    return test_string

def generate_text(llm, prompt, max_tokens=2000, temperature=0.7, max_retries=3, retry_delay=1):
    for i in range(max_retries):
        try:
            output = llm(prompt, max_tokens=max_tokens, temperature=temperature, stop=[], echo=False)
            response = remove_first_line(output['choices'][0]['text'].strip())
            print(f"Response: {response}")
            match = re.search(r'\[(.*?)\]', response)
            if match:
                try:
                    items = [item.strip().strip('"').strip("'") for item in match.group(1).split(',')]
                    print(f"Items: {items}")
                    return items
                except (SyntaxError, ValueError) as e:
                    print(f"Error parsing items: {e}")
                    pass
        except Exception as e:
            print(f"Error generating text (attempt {i+1}/{max_retries}): {e}")
            if i < max_retries - 1:
                time.sleep(retry_delay)
    return []

def search_web(search_term, api_key, cse_id):
    try:
        service = build("customsearch", "v1", developerKey=api_key)
        return service.cse().list(q=search_term, cx=cse_id).execute()
    except Exception as e:
        print(f"Error searching web: {e}")
        return None

def chunk_prompt(llm, prompt, max_tokens):
    prompt_bytes = prompt.encode('utf-8')
    prompt_tokens = llm.tokenize(prompt_bytes)
    chunks = []
    current_chunk = []
    for token in prompt_tokens:
        if len(current_chunk) + 1 <= max_tokens:
            current_chunk.append(token)
        else:
            chunks.append(llm.detokenize(current_chunk))
            current_chunk = [token]
    if current_chunk:
        chunks.append(llm.detokenize(current_chunk))
    return chunks

def process_chunks(llm, chunks, max_tokens):
    results = []
    for chunk in chunks:
        result = generate_text(llm, chunk, max_tokens=max_tokens)
        if isinstance(result, list):
            results.extend(result)
        else:
            results.append(result)
    combined_result = " ".join(results)
    return combined_result

def generate_subtopic_report(llm, research_topic, subtopic, search_cache, max_workers=5):
    all_queries = []
    search_data = []

    print(f"Generating initial search queries for subtopic: {subtopic}...")
    initial_queries_prompt = f"Generate 5 search queries to gather information on the subtopic '{subtopic}' of the research topic '{research_topic}'. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
    initial_queries = generate_text(llm, initial_queries_prompt)
    all_queries.extend(initial_queries)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        search_results = list(executor.map(lambda query: search_cache.setdefault(query, search_web(query, GOOGLE_API_KEY, GOOGLE_CSE_ID)), initial_queries))
    search_data.extend(search_results)

    print(f"Generating initial report for subtopic: {subtopic}...")
    report_prompt_1 = f"When writing your report, make it incredibly detailed, thorough, specific, and well-structured. Use Markdown for formatting. "
    report_prompt_2 = f"Analyze the following search data and generate a comprehensive report on the subtopic '{subtopic}' of the research topic '{research_topic}':\n\n{str(search_data)}"
    report_prompt = report_prompt_1 + report_prompt_2
    prompt_chunks = chunk_prompt(llm, report_prompt, max_tokens=4000)
    report = process_chunks(llm, prompt_chunks, max_tokens=4000)

    print(f"Analyzing report and generating additional searches for subtopic: {subtopic}...")
    analysis_prompt_1 = f"Analyze the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' and identify areas that need more detail or further information:\n\n{report}\n\n"
    analysis_prompt_2 = "---\n\nHere are all the search queries you have used so far for this subtopic:\n\n"
    analysis_prompt_3 = f"{str(all_queries)}\n\n---\n\nGenerate 3 new and unique search queries to fill in the gaps and provide more detail on the identified areas. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
    analysis_prompt = analysis_prompt_1 + analysis_prompt_2 + analysis_prompt_3
    additional_queries = generate_text(llm, analysis_prompt)
    all_queries.extend(additional_queries)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        additional_search_results = list(executor.map(lambda query: search_cache.setdefault(query, search_web(query, GOOGLE_API_KEY, GOOGLE_CSE_ID)), additional_queries))
    search_data.extend(additional_search_results)

    print(f"Updating report with additional information for subtopic: {subtopic}...")
    update_prompt_1 = f"Update the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' by incorporating the new information from the additional searches. Keep all existing information... only add new information:\n\n{report}\n\n---\n\nAdditional search data:\n\n{str(additional_search_results)}\n\n---\n\n"
    update_prompt_2 = "Generate an updated report that includes the new information and provides more detail in the identified areas. Use Markdown for formatting."
    update_prompt = update_prompt_1 + update_prompt_2
    update_prompt_chunks = chunk_prompt(llm, update_prompt, max_tokens=4000)
    report = process_chunks(llm, update_prompt_chunks, max_tokens=4000)

    print(f"Final report generated for subtopic: {subtopic}!")
    return report

def generate_comprehensive_report(llm, research_topic, subtopic_reports):
    print("Generating comprehensive report...")
    comprehensive_report_prompt_1 = f"Generate a comprehensive report on the research topic '{research_topic}' by combining the following reports on various subtopics:\n\n{subtopic_reports}\n\n"
    comprehensive_report_prompt_2 = "---\n\nEnsure that the final report is well-structured, coherent, and covers all the important aspects of the research topic. Make sure that it includes EVERYTHING in each of the reports, in a better structured, more info-heavy manner. Nothing -- absolutely nothing, should be left out. If you forget to include ANYTHING from any of the previous reports, you will face the consequences. Include a table of contents. Leave nothing out. Use Markdown for formatting."
    comprehensive_report_prompt = comprehensive_report_prompt_1 + comprehensive_report_prompt_2
    comprehensive_report_prompt_chunks = chunk_prompt(llm, comprehensive_report_prompt, max_tokens=4000)
    comprehensive_report = process_chunks(llm, comprehensive_report_prompt_chunks, max_tokens=4000)

    return comprehensive_report

def is_valid_subtopic_list(subtopic_list, research_topic):
    if not isinstance(subtopic_list, list):
        print("Subtopic checklist is not a list.")
        return False
    for subtopic in subtopic_list:
        if not isinstance(subtopic, str):
            print(f"Subtopic '{subtopic}' is not a string.")
            return False
        if subtopic.lower() == research_topic.lower():
            print(f"Subtopic '{subtopic}' is the same as the research topic.")
            return False
    return True

def main(research_topic, model_path, max_tokens=2000, max_subtopics=5, max_workers=5):
    try:
        llm = Llama(model_path=model_path)
    except Exception as e:
        print(f"Error loading Llama model: {e}")
        return

    if not GOOGLE_API_KEY:
        print("Google Search API key not found. Please set the GOOGLE_SEARCH environment variable.")
        return
    if not GOOGLE_CSE_ID:
        print("Google Custom Search Engine ID not found. Please set the GOOGLE_SEARCH_ENGINE_ID environment variable.")
        return

    subtopic_checklist_prompt = f"Generate a detailed checklist of subtopics to research for the topic '{research_topic}'. Return your checklist as a valid Python list, with each subtopic enclosed in single quotes and separated by commas. Maximum {max_subtopics} sub-topics. Start your response with ['"
    subtopic_checklist = generate_text(llm, subtopic_checklist_prompt)
    print(f"Generated subtopic checklist: {subtopic_checklist}")

    if not is_valid_subtopic_list(subtopic_checklist, research_topic):
        print("Invalid subtopic checklist generated. Please try again with a different research topic.")
        return

    print(f"Research Topic: {research_topic}")
    print(f"Subtopic Checklist: {subtopic_checklist}")

    search_cache = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        subtopic_reports = list(executor.map(lambda subtopic: generate_subtopic_report(llm, research_topic, subtopic, search_cache, max_workers=max_workers), subtopic_checklist))

    comprehensive_report = generate_comprehensive_report(llm, research_topic, "\n\n".join(subtopic_reports))
    print("Comprehensive report generated!")

    with open("comprehensive_report.txt", "w") as file:
        file.write(comprehensive_report)
    print("Comprehensive report saved as 'comprehensive_report.txt'.")

if __name__ == "__main__":
    research_topic = input("Enter the research topic: ")
    model_path = input("Enter the path to the Llama model file: ")
    main(research_topic, model_path, max_tokens=512, max_subtopics=5, max_workers=5)

In [ ]:
# was working for ~7 minutes then reaching context window post-subtopic research feelsbadman
# !pip install google-api-python-client
from googleapiclient.discovery import build
import concurrent.futures
import os
import re
from llama_cpp import Llama
import time

GOOGLE_API_KEY = os.getenv("GOOGLE_SEARCH")  # Replace with your Google Search API key
GOOGLE_CSE_ID = os.getenv("GOOGLE_SEARCH_ENGINE_ID")  # Replace with your Custom Search Engine ID

def remove_first_line(test_string):
    if test_string.startswith("Here") and test_string.split("\n")[0].strip().endswith(":"):
        return re.sub(r'^.*\n', '', test_string, count=1)
    return test_string

def generate_text(llm, prompt, max_tokens=2000, temperature=0.7, max_retries=3, retry_delay=1):
    for i in range(max_retries):
        try:
            output = llm(prompt, max_tokens=max_tokens, temperature=temperature, stop=[], echo=False)
            response = remove_first_line(output['choices'][0]['text'].strip())
            print(f"Response: {response}")
            match = re.search(r'\[(.*?)\]', response)
            if match:
                try:
                    items = [item.strip().strip('"').strip("'") for item in match.group(1).split(',')]
                    print(f"Items: {items}")
                    return items
                except (SyntaxError, ValueError) as e:
                    print(f"Error parsing items: {e}")
                    pass
        except KeyError as e:
            print(f"Error generating text (attempt {i+1}/{max_retries}): {e}")
            if i < max_retries - 1:
                time.sleep(retry_delay)
    return []

def search_web(search_term, api_key, cse_id):
    try:
        service = build("customsearch", "v1", developerKey=api_key)
        return service.cse().list(q=search_term, cx=cse_id).execute()
    except Exception as e:
        print(f"Error searching web: {e}")
        return None

def chunk_prompt(prompt, max_tokens):
    words = prompt.split()
    chunks = []
    current_chunk = []
    current_length = 0
    for word in words:
        word_length = len(word)
        if current_length + word_length + 1 <= max_tokens:
            current_chunk.append(word)
            current_length += word_length + 1
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = word_length + 1
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

def process_chunks(llm, chunks, max_tokens):
    results = []
    for chunk in chunks:
        result = generate_text(llm, chunk, max_tokens=max_tokens)
        results.append(result)
    combined_result = " ".join(results)
    return combined_result

def generate_subtopic_report(llm, research_topic, subtopic, search_cache, max_workers=5):
    all_queries = []
    search_data = []

    print(f"Generating initial search queries for subtopic: {subtopic}...")
    initial_queries_prompt = f"Generate 5 search queries to gather information on the subtopic '{subtopic}' of the research topic '{research_topic}'. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
    initial_queries = generate_text(llm, initial_queries_prompt)
    all_queries.extend(initial_queries)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        search_results = list(executor.map(lambda query: search_cache.setdefault(query, search_web(query, GOOGLE_API_KEY, GOOGLE_CSE_ID)), initial_queries))
    search_data.extend(search_results)

    print(f"Generating initial report for subtopic: {subtopic}...")
    report_prompt_1 = f"When writing your report, make it incredibly detailed, thorough, specific, and well-structured. Use Markdown for formatting. "
    report_prompt_2 = f"Analyze the following search data and generate a comprehensive report on the subtopic '{subtopic}' of the research topic '{research_topic}':\n\n{str(search_data)}"
    report_prompt = report_prompt_1 + report_prompt_2
    prompt_chunks = chunk_prompt(llm, report_prompt, max_tokens=4000)
    report = process_chunks(llm, prompt_chunks, max_tokens=4000)

    print(f"Analyzing report and generating additional searches for subtopic: {subtopic}...")
    analysis_prompt_1 = f"Analyze the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' and identify areas that need more detail or further information:\n\n{report}\n\n"
    analysis_prompt_2 = "---\n\nHere are all the search queries you have used so far for this subtopic:\n\n"
    analysis_prompt_3 = f"{str(all_queries)}\n\n---\n\nGenerate 3 new and unique search queries to fill in the gaps and provide more detail on the identified areas. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
    analysis_prompt = analysis_prompt_1 + analysis_prompt_2 + analysis_prompt_3
    additional_queries = generate_text(llm, analysis_prompt)
    all_queries.extend(additional_queries)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        additional_search_results = list(executor.map(lambda query: search_cache.setdefault(query, search_web(query, GOOGLE_API_KEY, GOOGLE_CSE_ID)), additional_queries))
    search_data.extend(additional_search_results)

    print(f"Updating report with additional information for subtopic: {subtopic}...")
    update_prompt_1 = f"Update the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' by incorporating the new information from the additional searches. Keep all existing information... only add new information:\n\n{report}\n\n---\n\nAdditional search data:\n\n{str(additional_search_results)}\n\n---\n\n"
    update_prompt_2 = "Generate an updated report that includes the new information and provides more detail in the identified areas. Use Markdown for formatting."
    update_prompt = update_prompt_1 + update_prompt_2
    update_prompt_chunks = chunk_prompt(llm, update_prompt, max_tokens=4000)
    report = process_chunks(llm, update_prompt_chunks, max_tokens=4000)

    print(f"Final report generated for subtopic: {subtopic}!")
    return report

def generate_comprehensive_report(llm, research_topic, subtopic_reports):
    print("Generating comprehensive report...")
    comprehensive_report_prompt_1 = f"Generate a comprehensive report on the research topic '{research_topic}' by combining the following reports on various subtopics:\n\n{subtopic_reports}\n\n"
    comprehensive_report_prompt_2 = "---\n\nEnsure that the final report is well-structured, coherent, and covers all the important aspects of the research topic. Make sure that it includes EVERYTHING in each of the reports, in a better structured, more info-heavy manner. Nothing -- absolutely nothing, should be left out. If you forget to include ANYTHING from any of the previous reports, you will face the consequences. Include a table of contents. Leave nothing out. Use Markdown for formatting."
    comprehensive_report_prompt = comprehensive_report_prompt_1 + comprehensive_report_prompt_2
    comprehensive_report_prompt_chunks = chunk_prompt(llm, comprehensive_report_prompt, max_tokens=4000)
    comprehensive_report = process_chunks(llm, comprehensive_report_prompt_chunks, max_tokens=4000)

    return comprehensive_report

def is_valid_subtopic_list(subtopic_list, research_topic):
    if not isinstance(subtopic_list, list):
        print("Subtopic checklist is not a list.")
        return False
    for subtopic in subtopic_list:
        if not isinstance(subtopic, str):
            print(f"Subtopic '{subtopic}' is not a string.")
            return False
        if subtopic.lower() == research_topic.lower():
            print(f"Subtopic '{subtopic}' is the same as the research topic.")
            return False
    return True

def main(research_topic, model_path, max_tokens=2000, max_subtopics=5, max_workers=5):
    try:
        llm = Llama(model_path=model_path)
    except Exception as e:
        print(f"Error loading Llama model: {e}")
        return

    if not GOOGLE_API_KEY:
        print("Google Search API key not found. Please set the GOOGLE_SEARCH environment variable.")
        return
    if not GOOGLE_CSE_ID:
        print("Google Custom Search Engine ID not found. Please set the GOOGLE_SEARCH_ENGINE_ID environment variable.")
        return

    subtopic_checklist_prompt = f"Generate a detailed checklist of subtopics to research for the topic '{research_topic}'. Return your checklist as a valid Python list, with each subtopic enclosed in single quotes and separated by commas. Maximum {max_subtopics} sub-topics. Start your response with ['"
    subtopic_checklist = generate_text(llm, subtopic_checklist_prompt)
    print(f"Generated subtopic checklist: {subtopic_checklist}")

    if not is_valid_subtopic_list(subtopic_checklist, research_topic):
        print("Invalid subtopic checklist generated. Please try again with a different research topic.")
        return

    print(f"Research Topic: {research_topic}")
    print(f"Subtopic Checklist: {subtopic_checklist}")

    search_cache = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        subtopic_reports = list(executor.map(lambda subtopic: generate_subtopic_report(llm, research_topic, subtopic, search_cache, max_workers=max_workers), subtopic_checklist))

    comprehensive_report = generate_comprehensive_report(llm, research_topic, "\n\n".join(subtopic_reports))
    print("Comprehensive report generated!")

    with open("comprehensive_report.txt", "w") as file:
        file.write(comprehensive_report)
    print("Comprehensive report saved as 'comprehensive_report.txt'.")

if __name__ == "__main__":
    research_topic = input("Enter the research topic: ")
    model_path = input("Enter the path to the Llama model file: ")
    main(research_topic, model_path, max_tokens=512, max_subtopics=5, max_workers=5)